<a href="https://colab.research.google.com/github/MOLINA46/Card/blob/main/AnalyticaPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Librerías
import pandas as pd
import plotly.express as px
import numpy as np
from google.colab import files
import io

# 2️⃣ Subir dataset
uploaded = files.upload()
archivo = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[archivo]))

# 3️⃣ Normalizar columnas y mostrar tabla inicial
df.columns = df.columns.str.strip().str.lower()
print("📊 Primeros registros del dataset:")
display(df.head())

# 4️⃣ Asegurar tipo datetime para fechas
if 'fecha_de_pago' in df.columns:
    df['fecha_de_pago'] = pd.to_datetime(df['fecha_de_pago'], errors='coerce')

# 5️⃣ Función para graficar y analizar
def plot_analysis(df):

    # --- Distribución de marcas ---
    if 'marca' in df.columns:
        marca_counts = df['marca'].value_counts().reset_index()
        marca_counts.columns = ['marca','cantidad']
        fig = px.bar(marca_counts, x='marca', y='cantidad', title='Distribución de marcas')
        fig.show()
        print(f"🔹 Marca más frecuente: {marca_counts.iloc[0]['marca']} ({marca_counts.iloc[0]['cantidad']} registros)")

    # --- Descuento por marca ---
    if 'marca' in df.columns and 'descuento' in df.columns:
        fig = px.box(df, x='marca', y='descuento', title='Distribución de descuento por marca')
        fig.show()

        # Outliers por marca
        for marca in df['marca'].unique():
            desc = df[df['marca']==marca]['descuento']
            q1, q3 = desc.quantile([0.25,0.75])
            iqr = q3 - q1
            outliers = desc[(desc < q1 - 1.5*iqr) | (desc > q3 + 1.5*iqr)]
            if len(outliers)>0:
                print(f"⚠️ Marca '{marca}' tiene {len(outliers)} descuentos atípicos: {outliers.values}")

    # --- Precio promedio por plaza ---
    if 'plaza' in df.columns and 'precio' in df.columns:
        df_avg = df.groupby('plaza', as_index=False)['precio'].mean().sort_values('precio', ascending=False)
        fig = px.bar(df_avg, x='plaza', y='precio', title='Precio promedio por plaza')
        fig.show()
        print(f"🔹 Plaza con mayor precio promedio: {df_avg.iloc[0]['plaza']} → {df_avg.iloc[0]['precio']:.2f}")

        # Rango de precios por plaza
        if 'precio' in df.columns:
            fig = px.box(df, x='plaza', y='precio', title='Rango de precios por plaza')
            fig.show()

    # --- Top 10 apellidos ---
    if 'apellido' in df.columns:
        top_apellidos = df['apellido'].value_counts().nlargest(10).reset_index()
        top_apellidos.columns = ['apellido','cantidad']
        fig = px.bar(top_apellidos, x='cantidad', y='apellido', orientation='h', title='Top 10 apellidos')
        fig.show()
        print(f"🔹 Apellido más frecuente: {top_apellidos.iloc[0]['apellido']} ({top_apellidos.iloc[0]['cantidad']} registros)")

    # --- Fechas de pago ---
    if 'fecha_de_pago' in df.columns:
        fig = px.histogram(df, x='fecha_de_pago', nbins=20, title='Distribución de fechas de pago')
        fig.show()
        df['month'] = df['fecha_de_pago'].dt.to_period('M')
        top_month = df['month'].value_counts().idxmax()
        print(f"🔹 Mes con más pagos: {top_month}")

    # --- Relación descuento vs precio ---
    if 'descuento' in df.columns and 'precio' in df.columns:
        fig = px.scatter(df, x='precio', y='descuento', color='marca' if 'marca' in df.columns else None,
                         title='Relación Descuento vs Precio', hover_data=df.columns)
        fig.show()

    # --- Descuento promedio por plaza ---
    if 'plaza' in df.columns and 'descuento' in df.columns:
        df_avg_desc = df.groupby('plaza', as_index=False)['descuento'].mean().sort_values('descuento', ascending=False)
        fig = px.bar(df_avg_desc, x='plaza', y='descuento', title='Descuento promedio por plaza')
        fig.show()
        print(f"🔹 Plaza con mayor descuento promedio: {df_avg_desc.iloc[0]['plaza']} → {df_avg_desc.iloc[0]['descuento']:.2f}")

# 6️⃣ Ejecutar análisis
plot_analysis(df)


Saving dataset_limpioYformateado.xlsx to dataset_limpioYformateado (4).xlsx
📊 Primeros registros del dataset:


,nombre_completo,nombre,apellido,marca,plaza,tarjeta_de_pago,fecha_de_pago,unnamed: 7,precio
0,HernándezCarlos,Carlos,Hernández,Stradivarius,Soria,3521423824467300,2024-08-06,0.991337,63.65
1,CarlosMolina,Juan,Molina,Mango,Las palmas,30435129159697,2024-03-04,0.702899,150.77
2,JorgeTorres,Jorge,Torres,Mango,Granada,4152975372852100096,2022-09-21,0.543179,292.72
3,LauraMolina,Laura,Molina,Mango,Cáceres,2321040928117710,2024-07-10,0.620072,62.78
4,TorresLucía,Lucía,Torres,Stradivarius,Badajoz,4734682976694330,2022-10-20,0.677546,209.88


🔹 Marca más frecuente: Mango (6 registros)


🔹 Plaza con mayor precio promedio: Valencia → 42025.90


🔹 Apellido más frecuente: Hernández (4 registros)


🔹 Mes con más pagos: 2022-09
